In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# reading files of all currencies and bitcoin:
bitcoin = pd.read_csv('../data/cleaned_data/bitcoin_clean.csv', index_col = 0)
dollar = pd.read_csv('../data/cleaned_data/dollar_clean.csv', index_col = 0)
euro = pd.read_csv('../data/cleaned_data/euro_clean.csv', index_col = 0)
yen = pd.read_csv('../data/cleaned_data/yen_clean.csv', index_col = 0)

In [3]:
bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1049 entries, 0 to 1048
Data columns (total 2 columns):
Date     1049 non-null object
Index    1049 non-null float64
dtypes: float64(1), object(1)
memory usage: 24.6+ KB


In [4]:
dollar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 748 entries, 0 to 747
Data columns (total 2 columns):
Date     748 non-null object
Index    748 non-null float64
dtypes: float64(1), object(1)
memory usage: 17.5+ KB


In [5]:
euro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 0 to 749
Data columns (total 2 columns):
Date     750 non-null object
Index    750 non-null float64
dtypes: float64(1), object(1)
memory usage: 17.6+ KB


In [6]:
yen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 750 entries, 0 to 749
Data columns (total 2 columns):
Date     750 non-null object
Index    750 non-null float64
dtypes: float64(1), object(1)
memory usage: 17.6+ KB


In [7]:
# creating a dataframe with all the values
currencies = bitcoin.set_index('Date').join(dollar.set_index('Date'), rsuffix='_usd', lsuffix = '_btc')\
.join(euro.set_index('Date')).join(yen.set_index('Date'), rsuffix='_yen', lsuffix='_eur')

In [8]:
currencies

,Index_btc,Index_usd,Index_eur,Index_yen
Date,,,,
2017-01-01,998.33,NaN,NaN,NaN
2017-01-02,1021.75,NaN,171.77,281.93
2017-01-03,1043.84,103.23,171.26,281.72
2017-01-04,1154.73,102.52,171.78,281.35
2017-01-05,1013.38,101.44,172.57,284.07
...,...,...,...,...
2019-11-11,8757.79,98.03,175.03,293.34
2019-11-12,8815.66,98.15,174.73,293.70
2019-11-13,8808.26,98.23,174.59,294.15


In [9]:
currencies.rename(columns={'Index_btc': 'BTC', 'Index_usd':'USD', 'Index_eur':'EUR', 'Index_yen':'JPY'}\
                  , inplace = True)

In [10]:
# as I want to compare values on same days, I don't want NaN values, but first I will check if there are many:
currencies.isnull().sum()

BTC      0
USD    306
EUR    304
JPY    304
dtype: int64

In [11]:
# I can see there are the same number of rows with NaN values in the 3 currencies in front of the Bitcoin. I will
# check where are they
currencies.query('USD == "NaN"')

,BTC,USD,EUR,JPY
Date,,,,
2017-01-01,998.33,NaN,NaN,NaN
2017-01-02,1021.75,NaN,171.77,281.93
2017-01-07,908.59,NaN,NaN,NaN
2017-01-08,911.20,NaN,NaN,NaN
2017-01-14,818.41,NaN,NaN,NaN
...,...,...,...,...
2019-10-27,9551.71,NaN,NaN,NaN
2019-11-02,9324.72,NaN,NaN,NaN
2019-11-03,9235.35,NaN,NaN,NaN


In [12]:
# The NaN values are due to non-opening sessions in the markets, so they are in bank holidays or weekends.
# I will drop that rows as I can't compare the values
currencies.drop(currencies.query('USD == "NaN"').index, axis = 0, inplace = True)

In [13]:
currencies.head()

,BTC,USD,EUR,JPY
Date,,,,
2017-01-03,1043.84,103.23,171.26,281.72
2017-01-04,1154.73,102.52,171.78,281.35
2017-01-05,1013.38,101.44,172.57,284.07
2017-01-06,902.20,102.20,172.77,282.24
2017-01-09,902.83,101.85,174.27,285.98


In [14]:
currencies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 743 entries, 2017-01-03 to 2019-11-15
Data columns (total 4 columns):
BTC    743 non-null float64
USD    743 non-null float64
EUR    743 non-null float64
JPY    743 non-null float64
dtypes: float64(4)
memory usage: 29.0+ KB


In [15]:
currencies.reset_index(inplace = True)

In [16]:
currencies.head()

,Date,BTC,USD,EUR,JPY
0,2017-01-03,1043.84,103.23,171.26,281.72
1,2017-01-04,1154.73,102.52,171.78,281.35
2,2017-01-05,1013.38,101.44,172.57,284.07
3,2017-01-06,902.20,102.20,172.77,282.24
4,2017-01-09,902.83,101.85,174.27,285.98


In [17]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [32]:
# creating a plot with a line for each currency. As BTC has values much higher than others, I use a secondary index.
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.update_layout(title="Bitcoin Evolution vs Currency Evolution",
    font=dict(family="Arial, monospace",size=18, color="#000000"))

fig.add_trace(go.Scatter(x=currencies['Date'], y=currencies['BTC'], name = 'BTC'), secondary_y=True)
fig.add_trace(go.Scatter(x=currencies['Date'], y=currencies['USD'], name = 'USD'))
fig.add_trace(go.Scatter(x=currencies['Date'], y=currencies['EUR'], name = 'EUR'))
fig.add_trace(go.Scatter(x=currencies['Date'], y=currencies['JPY'], name = 'JPY'))

In [19]:
# The plot shows big differences. To have some numbers, I will check some statistics:
currencies.describe()

,BTC,USD,EUR,JPY
count,743.000000,743.000000,743.000000,743.000000
mean,6267.900269,95.620767,180.665343,286.407254
std,3580.245614,3.063070,4.193385,8.581430
min,777.760000,88.500000,170.210000,271.290000
25%,3631.555000,93.580000,177.770000,280.595000
50%,6385.620000,96.010000,181.300000,284.650000
75%,8542.440000,97.480000,183.875000,290.565000
max,19114.200000,103.230000,188.960000,312.980000


In [20]:
"""
Acording to the plot and the statistics, I can say that in the last 3 years, BITCOIN has not behaved as a currency, 
or, at least, not as 3 of the most important currencies.
"""

'\nAcording to the plot and the statistics, I can say that in the last 3 years, BITCOIN has not behaved as a currency, \nor, at least, not as 3 of the most important currencies.\n'

In [21]:
# I will check some more stuff in TimeSeries

In [22]:
import statsmodels.tsa.api as sm

In [23]:
# as the data don't include weekends, I chose freq = 22 days, as working days.
btc_trend = sm.seasonal_decompose(currencies['BTC'], freq = 22).trend
usd_trend = sm.seasonal_decompose(currencies['USD'], freq = 22).trend
eur_trend = sm.seasonal_decompose(currencies['EUR'], freq = 22).trend
jpy_trend = sm.seasonal_decompose(currencies['JPY'], freq = 22).trend

In [31]:
# setting double indexes and plotting trends together
trends = make_subplots(specs=[[{"secondary_y": True}]])
trends.update_layout(title="Bitcoin Trend vs Currency Trends",
    font=dict(family="Arial, monospace",size=18, color="#000000"))

trends.add_trace(go.Scatter(x=currencies['Date'], y=btc_trend, name = 'BTC'), secondary_y=True)
trends.add_trace(go.Scatter(x=currencies['Date'], y=usd_trend, name = 'USD'))
trends.add_trace(go.Scatter(x=currencies['Date'], y=eur_trend, name = 'EUR'))
trends.add_trace(go.Scatter(x=currencies['Date'], y=jpy_trend, name = 'JPY'))

In [25]:
sm.seasonal_decompose(currencies['USD'], freq = 22).plot()


<Figure size 640x480 with 4 Axes>

In [33]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="Bitcoin Evolution",
    font=dict(family="Arial, monospace",size=18, color="#000000"))

fig.add_trace(go.Scatter(x=currencies['Date'], y=currencies['BTC'], name = 'BTC'))